In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo


$$
\begin{split}
E_x & = \sum_i^N \sum_j^N \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M c_{i,\mu} c_{j,\nu} c_{i,\kappa} c_{j,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_x & = \sum_i^N \sum_j^N  \left [ i j | i j \right ] \\
E_x & = \sum_i^N \sum_j^N  \langle  i j | j i  \rangle 
\end{split}
$$

In [2]:
def exchange_energy(Fouridx, C, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,N):
        for j in range(0,N):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            #print(i,j,mu,nu,kappa,lamda)
                            energy += C[mu,i]*C[nu,j]*C[kappa,i]*C[lamda,j]*Fouridx[mu,nu,kappa,lamda]

    return energy


$$
\begin{split}
E_H & = \sum_i^N \sum_j^N \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M c_{i,\mu} c_{j,\nu} c_{i,\kappa} c_{j,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_H & = \sum_i^N \sum_j^N  \left [ i i | j j \right ] \\
E_H & = \sum_i^N \sum_j^N  \langle  i j | i j \rangle 
\end{split}
$$

In [3]:
def hartree_energy(Fouridx, C, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,N):
        for j in range(0,N):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            #print(i,j,mu,nu,kappa,lamda)
                            energy += C[mu,i]*C[nu,i]*C[kappa,j]*C[lamda,j]*Fouridx[mu,nu,kappa,lamda]

    return energy


In [4]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #mol.basis = "ccpvdz", mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 in the case of  case 16 distinct elements
eri = mol.intor('int2e')
print(f"The System {mol.atom} has {eri.size} or {eri.shape[0]}^4 elements in 2-electron-intergrals/4-index-integrals matrix with the {mol.basis}-basis")



The System 
    He    0.    0.    0.
 has 16 or 2^4 elements in 2-electron-intergrals/4-index-integrals matrix with the 6-31g-basis


In [16]:
print("Overlap Integrals")
S = mol.intor('int1e_ovlp')
print(S)



## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

print("*"*24)
print("MO-Coefficent matrix")
for mu,AO in enumerate(mf.mo_coeff):
    print(f"Coefficients of mu={mu} {AO}")
print("*"*24)

#print("den ?")
#print(np.matmul(mf.mo_coeff.T,mf.mo_coeff))
#print("*"*24)


# get j, k and gamma (1RDM) matrix from hf, 
J = mf.get_j()
K = mf.get_k()
h = mf.get_hcore()
C = mf.mo_coeff
gamma = mf.make_rdm1()
print("Molpro-Gamma")
MPgamma=np.matmul(C,C.T)
for mu,AO in enumerate(MPgamma):
    print(f"Coefficients of mu={mu} {AO}")
print("Gamma")
for mu,AO in enumerate(gamma):
    print(f"Coefficients of mu={mu} {AO}")


Sm1 = np.linalg.inv(S)
# why do i have to multipl with S instead of S^{-1}
occu, naturalC = np.linalg.eigh(np.matmul(S,gamma))
print(f"should be natural occupation numbers, the sum of occupation numbers, i.e. N is {np.sum(occu)}")
print(occu)
print("*"*24)
print("Natural Orbital LC-Coefficent matrix")
for mu,AO in enumerate(naturalC):
    print(f"Coefficients of mu={mu} {AO}")
print("*"*24)
print(np.matmul(naturalC, naturalC.T))


Overlap Integrals
[[1.         0.63414774]
 [0.63414774 1.        ]]
converged SCF energy = -2.85516042615445
************************
MO-Coefficent matrix
Coefficients of mu=0 [ 0.59208126 -1.14981805]
Coefficients of mu=1 [0.51358601 1.1869588 ]
************************
Molpro-Gamma
Coefficients of mu=0 [ 1.67264177 -1.060702  ]
Coefficients of mu=1 [-1.060702    1.67264177]
Gamma
Coefficients of mu=0 [0.70112044 0.60816931]
Coefficients of mu=1 [0.60816931 0.52754118]
should be natural occupation numbers, the sum of occupation numbers, i.e. N is 2.0000000000000004
[-0.05635458  2.05635458]
************************
Natural Orbital LC-Coefficent matrix
Coefficients of mu=0 [ 0.67743651 -0.73558125]
Coefficients of mu=1 [-0.73558125 -0.67743651]
************************
[[1.00000000e+00 1.11249114e-17]
 [1.11249114e-17 1.00000000e+00]]


In [ ]:
# calculate the energy components to see what they are from the matrices the mf object offers you 
print(f"h_0 = {np.trace(np.matmul(h,gamma))} U = {1/2*np.trace(np.matmul(J,gamma))} E_x =  {1/4.*np.trace(np.matmul(K,gamma))}")
print(np.trace(np.matmul(h,gamma))+1/2.*np.trace(np.matmul(J,gamma))-1/4.*np.trace(np.matmul(K,gamma)))

# this should also work
print(f"U = {2*hartree_energy(eri, mf.mo_coeff, mol)} E_x = {exchange_energy(eri, mf.mo_coeff, mol)} ")